In [ ]:
from datasets import load_dataset

In [ ]:
# ローカル環境からデータセットをインポート

datasets = load_dataset(
    'text',
    data_files = {
        "train": "train.txt",
        "validation": "validation.txt"
    }
)

In [ ]:
model_name = "cyberagent/calm2-7b-chat"

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer_false = AutoTokenizer.from_pretrained(
    model_name,
    use_fast = False
)

In [ ]:
%%time
a = tokenizer_false(datasets["train"][0]['text'])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast = True
)

In [ ]:
%%time
b = tokenizer(datasets["train"][0]['text'])

↑
スピード比較

use_fast = Trueの方が僅かに速い？？

use_fastにすることで正確性を欠いたという報告もあり注意（[参考](https://github.com/huggingface/text-generation-inference/issues/469)）

In [ ]:
## 各データ単位のトークン化

def tokenize_function(examples):
    return tokenizer(examples['text'])

In [ ]:
tokenized_datasets = datasets.map(
    tokenize_function,
    batched = True, # トークン化の処理に複数バッチ使う
    num_proc = 4, # 並行プロセス数
    remove_columns = ['text'] # textデータは用済みなので削除
)

#### batched: バッチを使ったトークン化

データセットを特定のメモリサイズに落とし込む = 長い文章を短く分割する

大きめのバッチサイズを使うと処理が早く終わる傾向

ここでメモリサイズとはメモリを指すこともあるし、GPUを指すこともある

#### num_proc: マルチコア環境の場合

マルチコア環境 = 1つのCPUの中に複数の単位CPUがあるコンピュータ

マルチコア環境で実行している場合は、指定した数のCPUを使用する

並行処理することで高速化に寄与

コアの数よりも多く指定してしまうと、上限まで落とされる(警告が出る)

In [ ]:
# ファインチューニング中に使用するメモリ領域を圧迫しないように、適切なサイズに文章をカットする

block_size = 128

In [ ]:
def split_texts(examples):
    result = {
        'input_ids': [],
        'attention_mask': []
    }
    for ix in range(len(examples['input_ids'])):
        # 切り捨てを考慮してデータに含めるトークンIDsの数
        # カットサイズからはみ出る分のトークンIDは切り捨ててしまう、もったいないが
        total_length = len(examples['input_ids'][ix])
        total_lenght = (total_length // block_size) * block_size

        for bookmark in range(0, total_length, block_size):
            result['input_ids'].append(examples['input_ids'][ix][bookmark: bookmark + block_size])
            result['attention_mask'].append(examples['attention_mask'][ix][bookmark: bookmark + block_size])
    result["labels"] = result['input_ids'].copy() # transformersのファインチューニングのアルゴリズムによりlabelsとinput_idsが同じで問題ない(後述)
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    split_texts,
    batched = True,
    batch_size = 1000,
    num_proc = 4
)

#### labelsとinput_idsが同じで問題ない

hugging-faceのライブラリでファインチューニングする場合、labelsとinput_idsが同じで問題ない

実際のチューニングでは、1つのlabelsとinput_idsの組がなんども再利用される
というのも実際のチューニングアルゴリズムによるもので

途中まで生成された文章から次のトークンを予測して答え合わせ、というのを文章の最後まで行っているため

だから一つ前のlabelsが次のinput_idsになるというように一つずつずらしている

これをtransformersが気を効かせて勝手にやってくれる

## ここからファインチューニング

In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

↑

なんか若干容量の厳しさがあるかも

ファインチューニングと量子化の関係を調べる？

In [ ]:
output_dir = './results'
evaluation_strategy = 'epoch'
learning_rate = 2e-5
weight_decay = 0.01

In [ ]:
training_args = TrainingArguments(
    output_dir,
    evaluation_strategy = evaluation_strategy,
    learning_rate = learning_rate,
    weight_decay = weight_decay
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

In [ ]:
%%time
trainer.train()